In [1]:
using Pkg
Pkg.add("PowerModels")
Pkg.add("JuMP")
Pkg.add("Memento")
Pkg.add("Distributed")
Pkg.add("Clp")
Pkg.add("Ipopt")

   Updating registry at `C:\Users\Allen Vincent\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %[===========>                             ]  25.2 %>                           ]  30.4 %]  37.7 % %>            ]  68.3 % [================================>        ]  78.6 %0 %Fetching: [======================================>  ]  94.3 %

  Resolving package versions...
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Allen Vincent\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Allen Vincent

In [2]:
using PowerModels
using JuMP
using Memento
using Distributed
using Clp
using Ipopt

┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1260


Parsing the data using PowerModels.

In [274]:
network_data = PowerModels.parse_file("MatPowerData/case3.m")
pm = instantiate_model(network_data, ACPPowerModel, PowerModels.build_opf)

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 2 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 2 from 360.0 to 60.0 deg.
[warn | PowerModels]: no active generators found at bus 2, updating to bus type from 2 to 1
[warn | PowerModels]: active generators found at bus 3, updating to bus type from 1 to 2
[info | PowerModels]: removing 1 cost terms from generator 1: [100.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [100.0, 0.0]


ACPPowerModel(A JuMP Model
Minimization problem with:
Variables: 18
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 7 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`GenericQuadExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 15 constraints
Nonlinear: 8 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached., Dict{String,Any}("bus" => Dict{String,Any}("1" => Dict{String,Any}("zone" => 1,"bus_i" => 1,"bus_type" => 3,"vmax" => 1.05,"source_id" => Any["bus", 1],"area" => 1,"vmin" => 0.95,"index" => 1,"va" => 0.0,"vm" => 1.0…),"2" => Dict{St

Data that remains constant throughtout the algorithm.

In [275]:
"""
Constant data (for a network)

"""

# Number of generators 
num_gen = 0
for i in keys(network_data["gen"])
    num_gen+=1
end

# Number of buses
num_bus = 0
for i in keys(network_data["bus"])
    num_bus+=1
end

# Number of PQ, PV and reference buses
num_PQ = 0
num_PV = 0
num_ref = 0
for i in keys(network_data["bus"])
    if network_data["bus"][i]["bus_type"] == 1
        num_PQ+=1
    elseif network_data["bus"][i]["bus_type"] == 2
        num_PV+=1
    else
        num_ref=1
    end
end

# Number of branches
num_branch = 0
for i in keys(network_data["branch"])
   num_branch+=1 
end

# Edges at each bus (origin and destination)
f_loc = Array{Float64}(undef,num_branch)
t_loc = Array{Float64}(undef,num_branch)

for (i,branch) in ref(pm,:branch)
    f_loc[i] = branch["f_bus"]
    t_loc[i] = branch["t_bus"]
end

bus_f = []
bus_t = []

for (i,bus) in ref(pm,:bus)
    for j in 1:num_branch
        if i == f_loc[j]
            a = (i,j)
            push!(bus_f,a)
        elseif i == t_loc[j]
            a = (i,j)
            push!(bus_t,a)
        end
    end
end

# Generators at each bus
gen_loc = Array{Float64}(undef,num_gen)
for (i,gen) in ref(pm,:gen)
   gen_loc[i] = gen["gen_bus"]
end

bus_gen = []

for (i,bus) in ref(pm,:bus)
    for j in 1:num_gen
        if i == gen_loc[j]
            a = (i,j)
            push!(bus_gen,a) 
        end
    end
end

# Shunt Conductance and Susceptance
gs_bus = ones(num_bus)
bs_bus = ones(num_bus)
for i in keys(network_data["shunt"])
    index = network_data["shunt"][i]["index"]
    gs_bus[index] = network_data["shunt"][i]["gs"]
    bs_bus[index] = network_data["shunt"][i]["bs"]
end

# Demand at bus
pd_bus = zeros(num_bus)
qd_bus = zeros(num_bus)
for i in keys(network_data["load"])
    index = network_data["load"][i]["load_bus"]
    pd_bus[index] = network_data["load"][i]["pd"]
    qd_bus[index] = network_data["load"][i]["qd"]
end

Power Flow Step

In [276]:
# Power Flow Step
result_pf = PowerModels.compute_ac_pf(network_data)
PowerModels.update_data!(network_data, result_pf)
flows = PowerModels.calc_branch_flow_ac(network_data)
PowerModels.update_data!(network_data, flows)

result_pf["gen"]

Dict{String,Any} with 2 entries:
  "1" => Dict("qg"=>1.29239,"pg"=>4.35924)
  "2" => Dict("qg"=>2.85039,"pg"=>0.0)

In [277]:
"""
Data Wrangling: LP Step

"""
# Cost Arrays
d_cost = Array{Float64}(undef,num_gen)       # derivative of cost    
cost = Array{Float64}(undef,num_gen)         # cost
for i in keys(network_data["gen"])
    #(a,b,c) = network_data["gen"][i]["cost"]
    #index = network_data["gen"][i]["index"]
    #y = network_data["gen"][i]["pg"]
    #if y != 0
    #    d_cost[index] = (2*a*y + b)
    #    cost[index] = ((a*y*y)+(b*y)+ c)
    #else
    #    d_cost[index] = 0
    #    cost[index] = 0
    #end
    
    (a,b) = network_data["gen"][i]["cost"]
    index = network_data["gen"][i]["index"]
    y = network_data["gen"][i]["pg"]
    d_cost[index] = (a)
    cost[index] = ((a*y) + b)
end

# Line Flow Values
p_t = Array{Float64}(undef,num_branch)
p_f = Array{Float64}(undef,num_branch)
q_t = Array{Float64}(undef,num_branch)
q_f = Array{Float64}(undef,num_branch)
rate_a = Array{Float64}(undef,num_branch)
for i in keys(network_data["branch"])
    index = network_data["branch"][i]["index"]
    p_t[index] = network_data["branch"][i]["pt"]
    q_t[index] = network_data["branch"][i]["qt"]
    p_f[index] = network_data["branch"][i]["pf"]
    q_f[index] = network_data["branch"][i]["qf"]
    rate_a[index] = network_data["branch"][i]["rate_a"]
end

# Voltage magnitude and angle
v_m = Array{Float64}(undef,num_bus)
v_a = Array{Float16}(undef,num_bus)
for i in keys(network_data["bus"])
    index = network_data["bus"][i]["index"]
    v_m[index] = network_data["bus"][i]["vm"]
    v_a[index] = network_data["bus"][i]["va"]
end

# Values of real and reactive powers at pf solution
p_g = Array{Float64}(undef,num_gen)
q_g = Array{Float64}(undef,num_gen)
for i in keys(network_data["gen"])
    index = network_data["gen"][i]["index"]
    p_g[index] = network_data["gen"][i]["pg"]
    q_g[index] = network_data["gen"][i]["qg"]
end

In [278]:
susceptance = PowerModels.calc_susceptance_matrix(network_data)
bl = susceptance.matrix
admittance = PowerModels.calc_admittance_matrix(network_data)
yl = admittance.matrix
gl = real(yl)

sf = Array{Float64}(undef,num_branch)
cf = Array{Float64}(undef,num_branch)
st = Array{Float64}(undef,num_branch)
ct = Array{Float64}(undef,num_branch)
for (i,branch) in ref(pm,:branch)
    f = branch["f_bus"]
    t = branch["t_bus"]
    sf[i] = sin(v_a[f] - v_a[t])
    cf[i] = cos(v_a[f] - v_a[t])
    st[i] = sin(v_a[t] - v_a[f])
    ct[i] = cos(v_a[t] - v_a[f])
end


In [269]:
solver = JuMP.with_optimizer(Clp.Optimizer, PrimalTolerance=1e-6)

MathOptInterface.OptimizerWithAttributes(Clp.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("PrimalTolerance") => 1.0e-6])

In [281]:
"""
LP Step Formulation

"""
LP_step = Model(solver)


@variable(LP_step, pg[i = 1:num_gen])
@variable(LP_step, qg[i = 1:num_gen])

@variable(LP_step, pf[i = 1:num_branch])
@variable(LP_step, qf[i = 1:num_branch])
@variable(LP_step, pt[i = 1:num_branch])
@variable(LP_step, qt[i = 1:num_branch])

@variable(LP_step, vm[i = 1:num_bus])
@variable(LP_step, va[i = 1:num_bus])


# Voltage magnitude limit
for (i,bus) in ref(pm,:bus)
    index = bus["index"]
    @constraint(LP_step, vm[index] <= bus["vmax"]) 
    @constraint(LP_step, vm[index] >= bus["vmin"]) 
end

# Real and reactive power generation limits
for (i,gen) in ref(pm,:gen)
    index = gen["index"]
    @constraint(LP_step, pg[index] <= gen["pmax"]) 
    @constraint(LP_step, pg[index] >= 0)
    
end

# Voltage Angle
for (i,bus) in ref(pm,:bus)
    index = bus["index"]
    if bus["bus_type"] == 3
        @constraint(LP_step, va[index] == 0)
    end
end


# Thermal limit
for (i, branch) in ref(pm,:branch)
    index = branch["index"]
    @constraint(LP_step, 2*pf[index]*p_f[index] + 2*qf[index]*q_f[index] <= 2*rate_a[index]^2 + p_f[index]^2 + q_f[index]^2)
    @constraint(LP_step, 2*pt[index]*p_t[index] + 2*qt[index]*q_t[index] <= 2*rate_a[index]^2 + p_t[index]^2 + q_t[index]^2)
end


# Linearised KCL Constraints
set_g = []
set_f = []
set_t = []

for i in 1:num_bus
    for j in 1:num_gen
        if i == bus_gen[j][1]
            a = bus_gen[j][2]
            push!(set_g,a)  
        end
    end
    for k in 1:num_branch
        if i == bus_f[k][1]
            a = bus_f[k][2]
            push!(set_f,a)
        elseif i == bus_t[k][1]
            a = bus_t[k][2]
            push!(set_t,a)
        end
    end
    @constraint(LP_step, sum(pg[g] for g in set_g)  == 
                         sum(pf[f] for f in set_f) + pd_bus[i])
                        
    @constraint(LP_step, sum(qg[g] for g in set_g)  == 
                          sum(qf[f] for f in set_f) + sum(qt[t] for t in set_t) + qd_bus[i] - 
                             bl[i,i]*(2*vm[i]*v_m[i] + v_m[i]^2))
    set_g = []
    set_f = []
    set_t = []
end


for (i,branch) in ref(pm,:branch)
    f = branch["f_bus"]
    t = branch["t_bus"]
    
@constraint(LP_step, pf[i] == gl[f,f]*(2*vm[f]*v_m[f] - v_m[f]^2) + v_m[f]*v_m[t]*(bl[f,t]*sf[i] + gl[f,t]*cf[i]) +
                                  (vm[f] - v_m[f])*v_m[t]*(bl[f,t]*sf[i] + gl[f,t]*cf[i]) +
                                  (vm[t] - v_m[t])*v_m[f]*(bl[f,t]*sf[i] + gl[f,t]*cf[i]) +  
                                  (va[f] - v_a[f])*v_m[f]*v_m[t]*(bl[f,t]*cf[i] - gl[f,t]*sf[i]) +
                                  (va[t] - v_a[t])*v_m[f]*v_m[t]*(-bl[f,t]*cf[i] + gl[f,t]*sf[i]))
    
@constraint(LP_step, pt[i] == gl[t,t]*(2*vm[t]*v_m[t] - v_m[t]^2) + v_m[f]*v_m[t]*(bl[t,f]*st[i] + gl[t,f]*ct[i]) +
                                  (vm[f] - v_m[f])*v_m[t]*(bl[t,f]*st[i] + gl[t,f]*ct[i]) +
                                  (vm[t] - v_m[t])*v_m[f]*(bl[t,f]*st[i] + gl[t,f]*ct[i]) +  
                                  (va[f] - v_a[f])*v_m[f]*v_m[t]*(-bl[t,f]*ct[i] + gl[t,f]*st[i]) +
                                  (va[t] - v_a[t])*v_m[f]*v_m[t]*(bl[t,f]*ct[i] - gl[t,f]*st[i]))
    
@constraint(LP_step, qf[i] == -bl[f,f]*(2*vm[f]*v_m[f] - v_m[f]^2) + v_m[f]*v_m[t]*(gl[f,t]*sf[i] - bl[f,t]*cf[i]) +
                                  (vm[f] - v_m[f])*v_m[t]*(gl[f,t]*sf[i] - bl[f,t]*cf[i]) +
                                  (vm[t] - v_m[t])*v_m[f]*(gl[f,t]*sf[i] - bl[f,t]*cf[i]) +  
                                  (va[f] - v_a[f])*v_m[f]*v_m[t]*(gl[f,t]*cf[i] + bl[f,t]*sf[i]) +
                                  (va[t] - v_a[t])*v_m[f]*v_m[t]*(-gl[f,t]*cf[i] - bl[f,t]*sf[i]))
    
@constraint(LP_step, qt[i] == -bl[t,t]*(2*vm[t]*v_m[t] - v_m[t]^2) + v_m[f]*v_m[t]*(gl[t,f]*st[i] - bl[t,f]*ct[i]) +
                                  (vm[f] - v_m[f])*v_m[t]*(gl[t,f]*st[i] - bl[t,f]*ct[i]) +
                                  (vm[t] - v_m[t])*v_m[f]*(gl[t,f]*st[i] - bl[t,f]*ct[i]) +  
                                  (va[f] - v_a[f])*v_m[f]*v_m[t]*(-gl[t,f]*ct[i] - bl[t,f]*st[i]) +
                                  (va[t] - v_a[t])*v_m[f]*v_m[t]*(gl[t,f]*ct[i] + bl[t,f]*st[i]))
end


@objective(LP_step, Min, sum((d_cost[i]*(pg[i] - p_g[i])) + cost[i] for i=1:num_gen))

100 pg[1] + 100 pg[2]

In [282]:
optimize!(LP_step)
termination_status(LP_step)

Coin0506I Presolve 0 (-29) rows, 0 (-18) columns and 0 (-71) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00


OPTIMAL::TerminationStatusCode = 1

In [272]:
ref_bus = 0
for i in keys(network_data["bus"])
    if network_data["bus"][i]["bus_type"] == 3
        ref_bus = ref_bus = network_data["bus"][i]["index"]
    end
end

delta = Array{Float64}(undef, num_gen)
for i in 1:num_gen
    delta[i] = JuMP.value(pg[i])
    print(delta[i],"   ")
end

print("\n\n")
for i in keys(network_data["gen"])
    if network_data["gen"][i]["gen_bus"] != ref_bus
        index = network_data["gen"][i]["index"]
        network_data["gen"][i]["pg"] = delta[index]
        print(index,": ",network_data["gen"][i]["pg"], "     ")
    end
end


0.0   0.0   

2: 0.0     

In [273]:
delta_v = Array{Float64}(undef,num_bus)
for i in 1:num_bus
    delta_v[i] = JuMP.value(vm[i])
    print(delta_v[i],"   ")
end
print("\n\n")
for i in keys(network_data["bus"])
    index = network_data["bus"][i]["index"]
    if network_data["bus"][i]["bus_type"] == 2
        print(delta_v[index], "   ")
        network_data["bus"][i]["vm"] = delta_v[index]
        print(index,": ",network_data["bus"][i]["vm"], "     ")
    end
end

0.95   0.18632633619449063   0.95   

0.95   3: 0.95     

Loop back to the power flow step.